# 运行模型

In [1]:
import sys
import gc, csv
import numpy as np
import pandas as pd
sys.path.append("./global")
from helper import Timer, AnalysisCSV, ReadCSV, OFFLINE, CalQAuc, ORI_TRAIN_NAMES, ReduceMemUsage
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectPercentile, f_classif
from stage2.code.feature_selection import select_features
print(OFFLINE)

True


## 拼接特征

In [2]:
combine_all_features = True

In [3]:
base_feature_dir = "./stage1/output/"
base_npy_dir = "./stage2/input/"
recombine = True
if OFFLINE:
    csv_train_concat_feature_file = base_feature_dir + "debug_train_concat_feature.csv"
    npy_train_concat_feature_file = base_npy_dir + "debug_train_concat_feature.npy"
    npy_label_file = base_npy_dir + "debug_train_labels.npy"
    npy_train_auc_file = base_npy_dir + "debug_train_for_cal_auc.npy"
    train_file = "./stage1/input/train_last_50w.csv"
    feature_names = "./stage2/input/debug_feature_names.csv"
else:
    csv_train_concat_feature_file = base_feature_dir + "online_train_concat_feature.csv"
    csv_test_concat_feature_file = base_feature_dir + "online_test_concat_feature.csv"
    npy_train_concat_feature_file = base_npy_dir + "online_train_concat_feature.npy"
    npy_test_concat_feature_file = base_npy_dir + "online_test_concat_feature.npy"
    npy_train_auc_file = base_npy_dir + "online_train_for_cal_auc.npy"
    npy_label_file = base_npy_dir + "online_train_labels.npy"
    train_file = "./stage1/input/train_last_1000w.csv"
    feature_names = "./stage2/input/online_feature_names.csv"

### 修改合并文件列表

In [4]:
if OFFLINE:
    train_feature_files = [base_feature_dir + _ for _ in [
        "text_mining_feature/debug_trainTextMining_len.csv.gz",
        "text_mining_feature/debug_trainTextMining_ngramSim.csv.gz",
        # "vector_space/debug_train_vector_space_feature_lsi_dis.csv",
        # "vector_space/debug_train_vector_space_lsi_feature.csv"
        "word2vec/debug_trainWord2vec_senVecavgSim.csv.gz",
        "magic/debug_trainMagic_queryFreq.csv.gz",
        "magic/debug_trainMagic_diffSetSim.csv.gz",
        "magic/debug_trainMagic_ctrRateGlobal.csv.gz",
        "magic/debug_trainMagic_ctrRateLocal.csv.gz"
    ]]
else:
    train_feature_files = [base_feature_dir + _ for _ in [
        "text_mining_feature/online_trainTextMining_len.csv.gz",
        "text_mining_feature/online_trainTextMining_ngramSim.csv.gz",
        "word2vec/online_trainWord2vec_senVecavgSim.csv.gz",
        # "magic/online_trainMagic_ctrRateGlobal.csv.gz",
        "magic/online_trainMagic_ctrRateLocal.csv.gz",
        "magic/online_trainMagic_queryFreq.csv.gz",
        "magic/online_trainMagic_diffSetSim.csv.gz"
    ]]
    test_feature_files = [base_feature_dir + _ for _ in [
        "text_mining_feature/online_testTextMining_len.csv.gz",
        "text_mining_feature/online_testTextMining_ngramSim.csv.gz",
        "word2vec/online_testWord2vec_senVecavgSim.csv.gz",
        # "magic/online_testMagic_ctrRateGlobal.csv.gz",
        "magic/online_testMagic_ctrRateLocal.csv.gz",
        "magic/online_testMagic_queryFreq.csv.gz",
        "magic/online_testMagic_diffSetSim.csv.gz",
    ]]

In [5]:
# 合并所有特征
def CombineFeatures(feature_files, savefile, y=None):
    global feature_names
    # 拼接特征
    with Timer("concat features"):
        tmp = [ReduceMemUsage(ReadCSV(f, iterator=False)) for f in feature_files]
        features = pd.concat(tmp, axis=1)
        for _ in tmp:
            del _
        gc.collect()
    # change inf to nan
    with Timer("Change inf to nan"):
        features = features.astype(dtype=np.float32)
        inf = np.nan_to_num(np.inf)
        features = features.replace(inf, np.nan)
    # fill nan
    with Timer("Fill nan"):
        features = features.fillna(0.0)
    # 特征筛选
    with Timer("select features"):
        y = np.load(y)
        select_features("simple", features.values, y, features.columns.values, "./stage2/output/feature_selection/")
        select_features("ref", features.values, y, features.columns.values, "./stage2/output/feature_selection/")
        select_features("lgb", features.values, y, features.columns.values, "./stage2/output/feature_selection/")
        select_features("rf", features.values, y, features.columns.values, "./stage2/output/feature_selection/")
    # 存储header
    with open(feature_names, "w") as f:
        f.write(",".join(list(features.columns.values)))
    # 存储特征
    tmp1 = np.array(features)
    print("features shape:", tmp1.shape)
    print("two rows of features:")
    print(tmp1[:2])
    np.save(savefile, tmp1)
    print("feature file saved to "+savefile)
    del features, tmp1
    gc.collect()

### 拼接

In [19]:
# 拼接特征
if combine_all_features:
    CombineFeatures(train_feature_files, npy_train_concat_feature_file, npy_label_file)
    if not OFFLINE:
        CombineFeatures(test_feature_files, npy_test_concat_feature_file)

----->Started 'concat features' block...
Mem. usage decreased to  7.15 Mb (76.6% reduction)
Mem. usage decreased to 17.17 Mb (81.2% reduction)
Mem. usage decreased to  6.68 Mb (75.0% reduction)
Mem. usage decreased to  0.48 Mb (87.5% reduction)
Mem. usage decreased to  4.29 Mb (81.2% reduction)
Mem. usage decreased to  0.95 Mb (75.0% reduction)
Mem. usage decreased to  0.95 Mb (75.0% reduction)
----->Finished 'concat features' block, time used:6.25s.
----->Started 'Change inf to nan' block...
----->Finished 'Change inf to nan' block, time used:0.48s.
----->Started 'Fill nan' block...
----->Finished 'Fill nan' block, time used:0.11s.
----->Started 'select features' block...


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=5.4378e-11): result may not be accurate.
  overwrite_a=True).T
/opt/conda/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/opt/conda/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/opt/conda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/opt/conda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/opt/conda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1831: RuntimeWarning: inval

Save to ./stage2/output/feature_selection/features_weight_simple.csv
Save to ./stage2/output/feature_selection/features_weight_ref.csv


/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:741: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))


Training until validation scores don't improve for 40 rounds.
[50]	Dev's binary_logloss: 0.532989
Did not meet early stopping. Best iteration is:
[76]	Dev's binary_logloss: 0.532882
Training until validation scores don't improve for 40 rounds.


In [17]:
# tmp = ReadCSV("./stage2/output/feature_selection/features_weight_simple.csv", iterator=False)
# print(tmp)

In [31]:
!cat stage2/input/online_feature_names.csv

online_testTextMining_text_q-len,online_testTextMining_text_t-len,online_testTextMining_text_qt-len-radio,online_testTextMining_text_tq-len-radio,online_testTextMining_unigrams_q-len,online_testTextMining_unigrams_t-len,online_testTextMining_unigrams_qt-len-radio,online_testTextMining_unigrams_tq-len-radio,online_testTextMining_text_dice_ratio,online_testTextMining_text_edit_seq_ratio,online_testTextMining_text_edit_set_ratio,online_testTextMining_text_ratio,online_testTextMining_text_partial_ratio,online_testTextMining_text_token_sort_ratio,online_testTextMining_unigrams_dice_ratio,online_testTextMining_unigrams_edit_seq_ratio,online_testTextMining_unigrams_edit_set_ratio,online_testTextMining_unigrams_ratio,online_testTextMining_unigrams_partial_ratio,online_testTextMining_unigrams_token_sort_ratio,online_testTextMining_bigrams_dice_ratio,online_testTextMining_bigrams_edit_seq_ratio,online_testTextMining_bigrams_edit_set_ratio,online_testTextMining_bigrams_ratio,online_testTextMining

In [30]:
!cat stage2/input/debug_feature_names.csv

debug_trainTextMining_text_q-len,debug_trainTextMining_text_t-len,debug_trainTextMining_text_qt-len-radio,debug_trainTextMining_text_tq-len-radio,debug_trainTextMining_unigrams_q-len,debug_trainTextMining_unigrams_t-len,debug_trainTextMining_unigrams_qt-len-radio,debug_trainTextMining_unigrams_tq-len-radio,debug_trainTextMining_text_dice_ratio,debug_trainTextMining_text_edit_seq_ratio,debug_trainTextMining_text_edit_set_ratio,debug_trainTextMining_text_ratio,debug_trainTextMining_text_partial_ratio,debug_trainTextMining_text_token_sort_ratio,debug_trainTextMining_unigrams_dice_ratio,debug_trainTextMining_unigrams_edit_seq_ratio,debug_trainTextMining_unigrams_edit_set_ratio,debug_trainTextMining_unigrams_ratio,debug_trainTextMining_unigrams_partial_ratio,debug_trainTextMining_unigrams_token_sort_ratio,debug_trainTextMining_bigrams_dice_ratio,debug_trainTextMining_bigrams_edit_seq_ratio,debug_trainTextMining_bigrams_edit_set_ratio,debug_trainTextMining_bigrams_ratio,debug_trainTextMining

## 运行模型

In [18]:
! pip install  --index "http://pypi/simple" --trusted-host pypi  hyperopt
! pip install  --index "http://pypi/simple" --trusted-host pypi  pyemd

In [37]:
from stage2.code.run_hyperopt import LoadDataset, run_lgb_gbdt, run_lr

In [38]:
# 加载数据集
if OFFLINE:
    npy_test_concat_feature_file = None
LoadDataset(in_train_feature=npy_train_concat_feature_file, 
            in_train_label=npy_label_file, 
            in_test_feature=npy_test_concat_feature_file,
            in_q_auc_cal=npy_train_auc_file)

Feature names: ['online_testTextMining_text_q-len', 'online_testTextMining_text_t-len', 'online_testTextMining_text_qt-len-radio', 'online_testTextMining_text_tq-len-radio', 'online_testTextMining_unigrams_q-len', 'online_testTextMining_unigrams_t-len', 'online_testTextMining_unigrams_qt-len-radio', 'online_testTextMining_unigrams_tq-len-radio', 'online_testTextMining_text_dice_ratio', 'online_testTextMining_text_edit_seq_ratio', 'online_testTextMining_text_edit_set_ratio', 'online_testTextMining_text_ratio', 'online_testTextMining_text_partial_ratio', 'online_testTextMining_text_token_sort_ratio', 'online_testTextMining_unigrams_dice_ratio', 'online_testTextMining_unigrams_edit_seq_ratio', 'online_testTextMining_unigrams_edit_set_ratio', 'online_testTextMining_unigrams_ratio', 'online_testTextMining_unigrams_partial_ratio', 'online_testTextMining_unigrams_token_sort_ratio', 'online_testTextMining_bigrams_dice_ratio', 'online_testTextMining_bigrams_edit_seq_ratio', 'online_testTextMini

In [39]:
print(npy_train_concat_feature_file)
print(npy_label_file)
print(npy_test_concat_feature_file)
print(npy_train_auc_file)

./stage2/input/online_train_concat_feature.npy
./stage2/input/online_train_labels.npy
./stage2/input/online_test_concat_feature.npy
./stage2/input/online_train_for_cal_auc.npy


In [12]:
# import tqdm

In [18]:
run_lgb_gbdt("len_ngramSim_word2vecSim_all", eval_num=5)

{                                                  
 "bagging_fraction": 0.9,
 "bagging_freq": 6.0,
 "feature_fraction": 0.7000000000000001,
 "learning_rate": 0.18,
 "max_bin": 15,
 "min_data_in_leaf": 20.0,
 "num_iterations": 75.0,
 "num_leaves": 25.0,
 "num_threads": 4
}
[20]	valid's binary_logloss: 0.53349               
[40]	valid's binary_logloss: 0.532769              
[60]	valid's binary_logloss: 0.532459              
[20]	valid's binary_logloss: 0.533667              
[40]	valid's binary_logloss: 0.532895              
[60]	valid's binary_logloss: 0.532584              
[20]	valid's binary_logloss: 0.533705              
[40]	valid's binary_logloss: 0.533003              
[60]	valid's binary_logloss: 0.532706              
[20]	valid's binary_logloss: 0.533585              
[40]	valid's binary_logloss: 0.532822              
[60]	valid's binary_logloss: 0.532513              
[20]	valid's binary_logloss: 0.5335                
[40]	valid's binary_logloss: 0.532762             